In [ ]:
!pip install -q \
fastapi uvicorn pyngrok nest-asyncio geopy \
langchain-core langchain-community langchain-text-splitters langchain-huggingface\
langgraph \
transformers \
accelerate \
sentence-transformers \
faiss-cpu \
pydantic \
--upgrade
print('Pip installation completed !')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.2/109.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 

In [ ]:
from google.colab import drive
import json

# 1️⃣ Mount Google Drive
drive.mount('/content/drive')

# 2️⃣ Path to config JSON
CONFIG_FILE_PATH = "/content/drive/MyDrive/Agentic AI/config.json"

# 3️⃣ Load NGROK_AUTH_TOKEN from JSON
try:
    with open(CONFIG_FILE_PATH, "r") as f:
        config = json.load(f)
    NGROK_AUTH_TOKEN = config.get("NGROK_AUTH_TOKEN")
    if not NGROK_AUTH_TOKEN:
        raise ValueError("NGROK_AUTH_TOKEN missing in JSON")
except Exception as e:
    raise RuntimeError(f"Failed to load NGROK_AUTH_TOKEN: {e}")


# Cell 2: Ngrok V3 Update and Authtoken Configuration (Crucial Fixes)

# 1. Update ngrok binary to V3 (Fixes ERR_NGROK_121)
!sudo rm -f /usr/local/bin/ngrok
!wget -q -c -nc https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -qq -n ngrok-v3-stable-linux-amd64.zip
!mv ngrok /usr/local/bin/ngrok

# 2. Cleanup conflicting files (Fixes configuration conflicts)
!sudo rm -f /root/.ngrok2/ngrok.yml

# 3. Force your authtoken to be saved (Fixes ERR_NGROK_4018)
# Use the correct V3 command: 'ngrok authtoken <TOKEN>'
!/usr/local/bin/ngrok authtoken {NGROK_AUTH_TOKEN}

print("✅ Ngrok V3 configuration complete and authtoken saved.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ Ngrok V3 configuration complete and authtoken saved.


In [ ]:
# ====== Libraries ======
import sys
import random
import threading
import nest_asyncio
import time
import requests

from google.colab import drive

from typing import TypedDict, Dict, Any
from pydantic import BaseModel

from geopy.geocoders import Nominatim

from fastapi import FastAPI
import uvicorn
from pyngrok import ngrok

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline

from langgraph.graph import StateGraph, END

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [ ]:
# -----------------------
# CONFIG
# -----------------------
NGROK_AUTH_TOKEN = "" # <- Enter NGROK TOKEN
HF_TOKEN = "" # <- Enter HUGGING FACE TOKEN
API_PORT = random.randint(2000,9000)
print(f"**INTERNAL FASTAPI PORT:** {API_PORT}")
MODEL_NAME = 'google/flan-t5-large'
EMBEDDING_MODEL = 'all-MiniLM-L6-v2'

try:
  drive.mount('/content/drive')
  print('\nWaiting for file access....')
except Exception as e:
  print(f"Drive not mounted... {e}")

time.sleep(5)
KNOWLEDGE_FILE_PATH = "/content/drive/MyDrive/Agentic AI/knowledge_base.txt"
CONFIG_FILE_PATH = "/content/drive/MyDrive/Agentic AI/config.json"

# -----------------------
# LOAD TOKENS FROM JSON
# -----------------------
try:
    with open(CONFIG_FILE_PATH, "r") as f:
        config = json.load(f)
    NGROK_AUTH_TOKEN = config.get("NGROK_AUTH_TOKEN")
    HF_TOKEN = config.get("HF_TOKEN")
    print("Config loaded successfully!")
except FileNotFoundError:
    print(f"Config file not found at {CONFIG_FILE_PATH}")
    NGROK_AUTH_TOKEN = None
    HF_TOKEN = None
except json.JSONDecodeError:
    print(f"Error decoding JSON in {CONFIG_FILE_PATH}")
    NGROK_AUTH_TOKEN = None
    HF_TOKEN = None

# --- NEW FALLBACK KNOWLEDGE DEFINITION ---
FALLBACK_KNOWLEDGE_TEXT = """
URA Bank Loan Product Information

### Core Consumer Loans
* **Unsecured Personal Loan:**
    * Fixed Interest Rate: 6.5%
    * Maximum Tenure: 5 years
    * Maximum Loan Amount: $35,000
    * Prepayment Penalty: 1% of the remaining principal balance after the first year.

* **Home Purchase Loan:**
    * Fixed Interest Rate: 4.8% (for the first 3 years, then variable)
    * Maximum Tenure: 30 years
    * Maximum Loan Amount: $800,000
    * Prepayment Penalty: None.

### Operating Hours and Accounts
* **Main Bank Operating Hours:** Monday to Friday, **9:00 AM to 4:30 PM**. Saturday, **9:00 AM to 12:00 PM**.
* **IRA Accounts:** Individual Retirement Accounts (IRA) are only available to customers with a minimum balance of **$1,500** in a URA Bank checking or savings account.

### Specialized Secured Loans
* **Car Loan (New Vehicles Only):**
    * Maximum Amount: $50,000
    * Interest Rate: 5.5% fixed.
    * Prepayment Penalty: None.
"""
# --- END OF FALLBACK KNOWLEDGE DEFINITION ---

nest_asyncio.apply()

# -----------------------
# LOAD LLM
# -----------------------
print(f"\nLoading augmentation model: {MODEL_NAME}")
try:
  model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, token=HF_TOKEN)
  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=HF_TOKEN)

    pipe = pipeline(
      task='text2text-generation',
      model = model,
      tokenizer = tokenizer,
      max_new_tokens = 512,
      do_sample = True,
      temperature = 0.3,
      repetition_penalty = 1.5,
      model_kwargs = {'dtype':model.dtype,'eos_token_id':tokenizer.eos_token_id}
  )


  hf_llm = HuggingFacePipeline(
      pipeline = pipe,
      model_kwargs = {'stop':['###']},
  )
  print("LLM pipeline is ready.")
except Exception as e:
  print(f"Error loading model {MODEL_NAME}.... {e}")
  raise e

# -----------------------
# INGEST + VECTOR STORE
# -----------------------
def read_knowledge_text(path: str):
  try:
    with open(path, 'r', encoding='utf-8') as f:
      return f.read()
  except FileNotFoundError:
    print(f"Knowledge file not found at path: {path}. Using FallBack content")
    return FALLBACK_KNOWLEDGE_TEXT

def build_vector_store(text: str):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 380, chunk_overlap = 50)
  documents = [Document(page_content=chunk) for chunk in text_splitter.split_text(text)]
  embed_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
  vector_store = FAISS.from_documents(documents, embed_model)
  print(f"\n  --> Documents Indexed: {len(documents)}")
  return vector_store

knowledge_text = read_knowledge_text(KNOWLEDGE_FILE_PATH)
VECTOR_STORE = build_vector_store(knowledge_text)
print("\nVector Store is ready.")

retriever = VECTOR_STORE.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 2, 'fetch_k':8})

# -----------------------
# AUGMENT WITH LLM (Prompt)
# -----------------------

def augment_with_llm(context: str, query: str):
  prompt = f"""
You are a professional URA Bank assistant. Answer the Question **STRICTLY AND ONLY** using the Context below.
Your final answer must be **complete**, address **ALL parts** of the user's question, and use a structured format (headings or bullet points) for multi-part questions.

**INSTRUCTIONS:**
1. Use the Context to answer the Question fully.
2. Highlight key data points with **bold**.
3. Structure the output clearly using **bold headings** for each topic requested in the query. For comparisons, **ENSURE BOTH ITEMS ARE PRESENT AND SEPARATELY DESCRIBED**.
4. **DO NOT MENTION OR INCLUDE ANY TOPIC OR FACT NOT EXPLICITLY REQUESTED IN THE QUESTION.** For example, if the question asks about 'IRA Accounts,' do not mention 'Service Fees,' even if they appear in the same context block.
5. If not available, say: "I apologize, but the specific information you requested is not available in my knowledge base. Please contact a URA Bank specialist for further assistance."
6. Do not include any information that is not directly requested in the Question.
7. End with '###'.

>>> CONTEXT START <<<
{context}
>>> CONTEXT END <<<

Question: {query}
--------
Answer:
"""

  try:
    result = hf_llm.invoke(prompt).strip()
  except Exception as e:
    result = f"LLM Generation Failed: {e}."
  if result.endswith('###'):
    result = result[:-3].strip()
  return result

# -----------------------
# FASTAPI APP
# -----------------------
app = FastAPI(title="Bank RAG AI Agent")

@app.get("/")
def read_root():
  return {
      "status": "URA Bank AI Agent",
      "title": app.title,
      "description": app.description,
      "version": app.version,
      "api_docs": "Access Swagger UI at docs for full endpoint details."
  }

class QueryRequest(BaseModel):
  query: str

@app.post("/query_index", response_model=Dict[str,Any])
async def query_index(request: QueryRequest):
  query = request.query
  try:
    print("First attempt to get relevent information from Vector Store")
    docs = retriever.invoke(query)
  except Exception as e:
    print(f"Second attempt to get relevent information from Vector Store, due to Exception {e}")
    docs = retriever.get_relevenat_documents(query)
  context = "\n----\n".join([d.page_content for d in docs])
  print(f"Debug: Retrieved Context for {query[:20]}....': {context[:100]}....")
  final_answer = augment_with_llm(context, query)
  return {"query":query, "index_response":final_answer, 'llm_model': MODEL_NAME}


@app.get('/get_branch_info')
def get_branch_info(zip_code: str = '500034'):
  geolocator = Nominatim(user_agent="ura_bank_locator")
  search_query = f"HDFC Bank {zip_code}"
  try:
    location = geolocator.geocode(search_query)
    if location:
      address = f"URA Bank Branch (Nearest  to {zip_code}): {location.address}"
    else:
      address = address = f"URA Bank Branch not found for zip code {zip_code}."
  except Exception as e:
    address = f"Geocoding failed: {e}"
  return {"branch_address": address}

# -----------------------
# LANGGRAPH AGENT
# -----------------------
class AgentState(TypedDict):
  query: str
  rag_result: str


def tool_node(state: AgentState):
  url = f"http://127.0.0.1:{API_PORT}/query_index"
  print(f"\nLangGraph: Calling Rag tool for query: {state['query'][:50]}....")
  response = requests.post(url, json={'query':state['query']},timeout=300)
  if response.status_code == 200:
    result = response.json().get('index_response','Error in RAG response')
  else:
    result = f"RAG tool failed with status {response.status_code}"
  return AgentState(query=state['query'], rag_result=result)

def synthesis_node(state: AgentState):
  return AgentState(
      query = state['query'],
      rag_result =(
      f"**FINAL REPORT from LangGraph**\n"
      f"**Query:** {state['query']}\n"
      f"**Answer (via {MODEL_NAME} Augmentation):** {state['rag_result']}"
      )
  )

# -----------------------
# SERVER + NGROK
# -----------------------
def run_server():
  uvicorn.run(app, host="127.0.0.1", port=API_PORT, log_level='error')

def start_ngrok(port=API_PORT):
  try:
    if NGROK_AUTH_TOKEN:
      ngrok.set_auth_token(NGROK_AUTH_TOKEN)
      tunnel = ngrok.connect(port)
      url = tunnel.public_url
      print("\n"+ "=="*80)
      print(f'FastAPI is live on PUBLIC ULR: {url}')
      print(f'Swagger UI                   : {url}/docs')
      print("\n"+ "=="*80)
      return tunnel
  except Exception as e:
    print(f'NGROK set up failed. {e}')
  return None

if __name__ == '__main__':
  server_thread = threading.Thread(target=run_server, daemon=True)
  server_thread.start()
  print(f'\nWaiting 15s for uvicorn server to stabilize....')
  time.sleep(15)
  tunnel_object = start_ngrok(API_PORT)

  graph_builder = StateGraph(AgentState)
  graph_builder.add_node('rag_tool',tool_node)
  graph_builder.add_node('final_summary', synthesis_node)
  graph_builder.set_entry_point('rag_tool')
  graph_builder.add_edge('rag_tool', 'final_summary')
  graph_builder.add_edge('final_summary',END)
  graph = graph_builder.compile()

  test_queries = [
        "What is the fixed interest rate and the maximum tenure for the Unsecured Personal Loan compared to the Home Purchase Loan?",
        "What are the main operating hours for the bank and the requirement for IRA accounts?",
        "What is the maximum loan amount and interest rate for a Car Loan, and is there a prepayment penalty?",
    ]

  for q in test_queries:
    print(f"\n-- Query: {q[:50]}")
    final_state = graph.invoke({"query":q},config={'recursion_limit':10},timeout=300)

    print("\n" + "=" * 80)
    print(final_state['rag_result'])
    print("=" * 80 + "\n")

  print("\n--- Geopy Tool Test ---")
  try:
    res = requests.get(f"http://127.0.0.1:{API_PORT}/get_branch_info?zip_code=500034")
    print(f"Branch address: {res.json().get('branch_address')}")
  except Exception as e:
    print(f"Geopy failed: {e}")


  try:
    print("\nServer running. Press Ctrl+C to stop Uvicorn and close Ngrok tunnel.")

    while True:
      time.sleep(1)

  except KeyboardInterrupt:
    if tunnel_object:
      ngrok.kill()
    print("\nShutdown complete.")


**INTERNAL FASTAPI PORT:** 5138
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Waiting for file access....

Loading augmentation model: google/flan-t5-large


Device set to use cpu


LLM pipeline is ready.

  --> Documents Indexed: 8

Vector Store is ready.

Waiting 15s for uvicorn server to stabilize....

FastAPI is live on PUBLIC ULR: https://autoicous-daine-homelike.ngrok-free.dev
Swagger UI                   : https://autoicous-daine-homelike.ngrok-free.dev/docs


-- Query: What is the fixed interest rate and the maximum te

LangGraph: Calling Rag tool for query: What is the fixed interest rate and the maximum te....
First attempt to get relevent information from Vector Store
Debug: Retrieved Context for What is the fixed in....': --- URA Bank Loan Product Information ---

### Core Consumer Loans
The fixed interest rate for the *....

**FINAL REPORT from LangGraph**
**Query:** What is the fixed interest rate and the maximum tenure for the Unsecured Personal Loan compared to the Home Purchase Loan?
**Answer (via google/flan-t5-large Augmentation):** ### The fixed interest rate for the **Unsecured Personal Loan** is **7.5%** with a maximum tenure of **60 months**